##### App Basics

Importing Packages

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
from pandas_datareader import wb
from datetime import datetime as dt

Running and Hosting via Dash

In [2]:
# Initialize the app
app = JupyterDash()
type(app)

jupyter_dash.jupyter_app.JupyterDash

In [7]:
# Layout initialisation
app.layout = html.Div(
    children = [
        html.H1('application title here'),  # sample title
        html.P('description of application here'),   # sample description

        dcc.Graph(figure = myFig,  style = {'marginLeft' : 200, 'marginRight' : 200}), # graph object

        html.Div('This is the first inner division', style = {'backgroundColor' : 'yellow'}), # style for section 1
        
        html.Br(), # line break

        html.Div('This is the second inner division', style = {'backgroundColor' : 'red'}),   # style for section 2
        
        html.Br(), # line break
        
        html.Div('This is the third inner division', style = {'backgroundColor' : 'green'}),   # style for section 3

        dcc.Markdown(""" Markdown Text uses three quotation marks. """),    # Markdown text

        html.Div(dcc.Markdown(""" You can use variables to store these. """), style = {'color' : 'white', 'backgroundColor' : 'black'}), # Markdown text with style
        ],

    style = { # all style elements are written in camelCase, while the properties are written in kebab-case
        'textAlign' : 'center',            # center-align all text
        'backgroundColor' : 'black',       # change background color using a color string
        'color' : '#7FDBFF',               # change text color using a Hex color code
    },
)

In [ ]:
# running a server
app.run_server(mode='inline')

##### Selectors, Hamburger Menu, etc.

In [13]:
# List of dictionaries with key-value pairs
myOptions = [
    {'label' : 'Life expectancy', 'value' : 'life_exp'},
    {'label' : 'Internet users', 'value' : 'internet'},
    {'label' : 'Population', 'value' : 'pop'}
]

# Store Dropdown component in a variable
metric = dcc.Dropdown(
    options = myOptions,    # list of dicts with label-value pairs
    value = 'life_exp',       # default value (value not label)
    multi = True             # allows multiple selections
)

Selector in Dash App

In [ ]:
app = JupyterDash()

app.layout = html.Div(
    children = [
        
        # Header
        html.H1('Hello Dash!'),                                        
        html.P('Dash: A web application framework for your data.'),
        
        # Dropdown menu
        html.Label('Choose metric:'),
        metric
    ]
)

app.run_server()

##### Multiselector: Countries from WB, Radio Buttons, Checklist

In [2]:
# Indicator getter
def worldBank(indDict, startYear, endYear):
    # Download the data
    df = wb.download(indicator = indDict.keys(), 
                        country = 'all', 
                        start = startYear, 
                        end = endYear).reset_index()

    df.rename(columns = indDict, inplace = True) # rename columns
    df['year'] = df['year'].astype(int) # unstringify the year

    # Unaggregate the data, cus we don't like aggregates
    dfInfo = wb.get_countries() # get country info
    dfInfo = dfInfo[dfInfo['region'] != 'Aggregates'].copy() # remove aggregates
    dfInfo.rename(columns = {'name' : 'country'}, inplace = True) # rename column
    df = df.merge(dfInfo, on = 'country', how = 'inner') # merge the two dataframes

    # Return the dataframe
    return df

In [3]:
# get dataframe
indDict = {'SP.DYN.LE00.IN' : 'life_exp',
            'IT.NET.USER.ZS' : 'internet',
            'SP.POP.TOTL'    : 'pop'}
dfWB = worldBank(indDict, 2000, 2020)

# radio item options
options = [
    {'label' : 'Life expectancy', 'value' : 'life_exp'},
    {'label' : 'Internet users', 'value' : 'internet'},
    {'label' : 'Population', 'value' : 'pop'}
]

# radio item
buttons = dcc.RadioItems(
    options = options, 
    value = 'life_exp'
)

check_list = dcc.Checklist(
    options = options,
    value = ['life_exp']
)

In [4]:
# create menu
countries = [{'label' : country, 'value' : country} for country in dfWB['country'].unique()]

country_menu = dcc.Dropdown(
            options = [{'label' : country, 'value' : country} for country in dfWB['country'].unique()],
            value = 'Norway'
        )


In [ ]:
app = JupyterDash()

app.layout = html.Div(
    children = [
        
        # Header
        html.H1('Hello Dash!'),                                        
        html.P('Dash: A web application framework for your data.'),
        
        # Insert single drowdown
        html.Label('Choose countries:'),
        country_menu,

        # Radio buttons
        html.Label('Choose metric:'),
        buttons,

        # Insert check list
        html.Label('Choose multiple metrics:'),
        check_list
    ]
)

app.run_server(mode='inline')

##### Datetime, Date Selection

In [ ]:
single_date = dcc.DatePickerSingle(
    min_date_allowed = dt(2020, 1, 1),        # minimum date user can select
    max_date_allowed = dt(2020, 12, 31),      # maximum date user can select
    date = dt(2020, 8, 20)                    # initial date
)

range_date = dcc.DatePickerRange(
    min_date_allowed = dt(2020, 1, 1),        # minimum date user can select
    max_date_allowed = dt(2020, 12, 31),      # maximum date user can select
    start_date = dt(2020, 8, 20),             # initial start date
    end_date = dt(2020, 8, 27)                # initial end date
)

##### Worldbank Related

In [33]:
# Indicator getter
def worldBank(indDict, startYear, endYear):
    # Download the data
    df = wb.download(indicator = indDict.keys(), 
                        country = 'all', 
                        start = startYear, 
                        end = endYear).reset_index()

    df.rename(columns = indDict, inplace = True) # rename columns
    df['year'] = df['year'].astype(int) # unstringify the year

    # Unaggregate the data, cus we don't like aggregates
    dfInfo = wb.get_countries() # get country info
    dfInfo = dfInfo[dfInfo['region'] != 'Aggregates'].copy() # remove aggregates
    dfInfo.rename(columns = {'name' : 'country'}, inplace = True) # rename column
    df = df.merge(dfInfo, on = 'country', how = 'inner') # merge the two dataframes

    # Return the dataframe
    return df

# Using it:
# you can find the tickers here: https://data.worldbank.org/indicator
indDict = {'SP.DYN.LE00.IN' : 'life_exp',
            'IT.NET.USER.ZS' : 'internet',
            'SP.POP.TOTL'    : 'pop'}

dfWB = worldBank(indDict, 2000, 2020)
dfWB.head()

,country,year,life_exp,internet,pop,iso3c,iso2c,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,Afghanistan,2020,62.575,18.4,38972230.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
1,Afghanistan,2019,63.565,17.6,37769499.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
2,Afghanistan,2018,63.081,16.8,36686784.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
3,Afghanistan,2017,63.016,13.5,35643418.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
4,Afghanistan,2016,63.136,11.0,34636207.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228


Scatter Plot

In [29]:
def scatterWB(df, countryList, x, y, size, color, xLim, yLim):
    copyFrame = df.copy()
    copyFrame.sort_values(['year'], inplace = True)
    copyFrame['label'] = ''

    for country in countryList:
        copyFrame.loc[copyFrame['country'] == country, 'label'] = country
    
    fig = px.scatter(copyFrame,
                    x = x, # f.e. internet
                    y = y, # f.e. life_exp
                    size = size, # f.e. pop
                    color = color, # f.e. region
                    hover_name = 'country', # f.e. country
                    text = 'label', # f.e. country
                    animation_frame = 'year',
                    animation_group = 'country', # f.e. country
                    range_x = xLim, # adjust range to your data
                    range_y = yLim, # adjust range to your data
    )

    fig.update_traces(textposition = 'top center')

    fig.update_layout(
        xaxis_title = x,
        yaxis_title = y,
        title = 'title here',
        showlegend = False,
    )

    return fig

# Using it:
myFig = scatterWB(dfWB, ['Norway'], x='internet', y='life_exp', size='pop', color='region', xLim=[-5, 105], yLim=[35, 90])
myFig.show()

##### Plotly Express

In [ ]:
import plotly.express as px
from pandas_datareader import wb

Line Chart

In [ ]:
# Create figure object and store in variable
fig = px.line(
    df,          # dataframe
    y = 'Close',     # column on y-axis
    width = 1000,    # width in pixels
    height = 500,    # height in pixels
)

fig.update_layout(
    yaxis_title = None,                         # remove label on y-axis  
    xaxis_title = None,                         # remove label on x-axis
    title = 'Daily closing price of Bitcoins',  # add title
    title_x = 0.5,                              # location of title
)

fig.update_xaxes(
    rangeslider_visible = True # add range slider
)  

In [ ]:
def add_rangeselector(figure):
    """This function takes a plotly figure object and adds buttons to the figure that allows the user
    to zoom in on several pre-defined periods. Source: https://plotly.com/python/range-slider/."""
    
    figure.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(count=1,
                             label="YTD",
                             step="year",
                             stepmode="todate"),
                        dict(count=1,
                             label="1y",
                             step="year",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            )
        )

Functional Form

In [ ]:
def lineCovid(df, countries):
    subset = df[df['location'].isin(countries)]

    fig = px.line(
        subset,
        x = 'date',
        y = 'total_deaths',
        color = 'location'
    )

    fig.update_layout(
        yaxis_title = None,
        xaxis_title = None,
        title = 'Total number of covid deaths',
        title_x = 0.5,
        legend_title_text = None
    )

Scatter Plot

In [ ]:
fig = px.scatter(
    df,
    x = 'internet',
    y = 'life_exp',
    size = 'pop',            # adjust marker size for each observation
    hover_name = 'country',  # add title for each observation in the hover label
    #text = 'country',       # ...or add label directly to each observation
)

fig.update_layout(
    yaxis_title = 'Life expectency at birth (yrs)',
    xaxis_title = 'Share of population with access to internet (%)',
    legend_title_text = None,
    legend_x = 0,                             # adjust position of legend on the x-axis
    legend_y = 1.15,                          # adjust position of legend on the y-axis
    legend_orientation = 'h',                 # make legend horizontal
)

In [ ]:
fig = px.scatter(
    df_wb_country,
    x = 'internet',
    y = 'life_exp',
    size = 'pop',
    color = 'region',
    hover_name = 'country',
    animation_frame = 'year',      # the time variable is years... 
    animation_group = 'country',   # ...and the units that we are tracking over time are countries
    range_x = [-5, 105],           # fix range on x-axis
    range_y = [35, 90],            # fix range on y-axis
)

fig.update_layout(
    yaxis_title = 'Life expectency at birth (yrs)',
    xaxis_title = 'Share of population with access to internet (%)',
    showlegend = False,
)


fig.update_traces(textposition='top center') # change position of label relative to the marker

Bar Plot

In [ ]:
fig = px.bar(
    df_covid_sum, 
    x = 'date', 
    y = 'new_deaths'
)   

fig.update_layout(
    xaxis_title = None, 
    yaxis_title = None,
    title = 'Weekly number of COVID-19 deaths in the world',
    title_x = 0.5,
    showlegend = False
)

In [ ]:
# use df_wb_regions to create a bar plot of the internet access in 2018
fig = px.bar(
    df_wb_regions[df_wb_regions['year'] == 2018],
    x = 'region',
    y = 'internet',
    color = 'region',
    color_discrete_sequence = px.colors.qualitative.Pastel,
    title = 'Share of population with access to internet (%)',
    labels = {'internet' : '%'},
    hover_data = {'region' : False}
)

#show plot
fig.show()

Stacked Bar

In [ ]:
# Extract continents
continents = ['Africa', 'Europe', 'Asia', 'North America', 'South America']
df_covid_regions = df_covid[df_covid['location'].isin(continents)]

# Set index
df_covid_regions.set_index('date', inplace = True)

# Groupby continents BEFORE resample to week level
df_covid_regions = df_covid_regions.groupby('location')['new_deaths'].resample('W').sum().reset_index()

df_covid_regions.head()

In [ ]:
fig = px.bar(
    df_covid_regions,
    x = 'date',         
    y = 'new_deaths',
    color = 'location',  # specify the column to determine the groups
)   

fig.update_layout(
    xaxis_title = None, 
    yaxis_title = None,
    title = 'Weekly number of COVID-19 deaths in the world',
    title_x = 0.5,
    legend_title = 'Continent',
)

In [ ]:
# bar plot of the internet access from 2000 to 2015 by region
fig = px.bar(
    df_wb_regions[df_wb_regions['year'].isin(range(2000, 2016))],
    x = 'year',
    y = 'internet',
    color = 'region',
    color_discrete_sequence = px.colors.qualitative.Pastel,
    title = 'Share of population with access to internet (%)',
    labels = {'internet' : '%'},
    hover_data = {'region' : False}
)

#show plot
fig.show()

Maps

In [ ]:
# Extract subset
year = 2019
subset = df_wb_country[df_wb_country['year'] == year].copy()

fig = px.choropleth(
    subset,                               # dataframe
    locations = 'iso3c',                  # column with country codes
    color = 'internet',                   # column to color the map with
    color_discrete_sequence = px.colors.qualitative.Pastel,    # colorscale
    hover_name = 'country',              
    hover_data = {'iso3c' : False},
)
       
fig.update_layout(
    title = 'Share of population with access to internet, ' + str(year),
    title_x = 0.5,
    coloraxis_colorbar_title = 'percent',
    geo_showframe = False,
)

In [ ]:
subset = df_wb_country[df_wb_country['year'] == 2019]

fig = px.scatter_geo(
    subset, 
    locations = 'iso3c',
    size = 'pop',               # column label to weight the markers with
)

fig.update_layout(
    title = 'Population, 2019',
    title_x = 0.5,
    geo_showcountries = True,   # set to True in order to see country borders
)

##### Styling APP

In [ ]:
description = """
A Dash application that plots the daily closing price for a given ticker.

Data is collected from [Yahoo! finance](https://finance.yahoo.com/).
"""

app.layout = html.Div(
    children = [
        
        # Header
        html.H1('Stock explorer'),
        dcc.Markdown(description),
        
        # Insert graph
        dcc.Graph(figure = fig_close, style = {'marginLeft' : 100, 'marginRight' : 100})
    ],
    
    style = {
        'textAlign' : 'center', 
        'backgroundColor' : 'black', 
        'color' : 'white'
    }
)

##### Callbacks

In [ ]:
from dash.dependencies import Input, Output

In [ ]:
# Syntax
@app.callback(
    Output(component_id, component_property),
    Input(component_id, component_property)
)

In [ ]:
app = JupyterDash(external_stylesheets = [dbc.themes.BOOTSTRAP, dbc_css])

app.layout = dbc.Container(
    children = [
        
        # Header
        html.H1('Hello Dash!'),
        html.P('Dash: A web application framework for your data.'),
        
        # Input component
        dcc.RadioItems(
            options = options, 
            value = 'pop',     # component_property for input
            id = 'my_input',   # component_id for input                       
        ),
        
        # Output component
        html.H3(
            children = '',    # component_property for output (the empty string is just a placeholder)          
            id = 'my_output', # component_id for output
        )
    ],
    className = 'dbc'

)

@app.callback(
    Output(component_id = 'my_output', component_property = 'children'), # link the output from the function with html.H3
    Input(component_id = 'my_input', component_property = 'value')       # link the input to the function with dcc.RadioItems
) 

def update_metric(choice):
    return 'You have chosen "' + str(choice) + '"!'

app.run_server()